# Tutorial 2: Loading and MM Preparation

MiMiCPy is a python wrapper for preparing and executing MiMiC runs. For details on installation and compatibility, please refer to https://github.com/bharurn/mimicpy.

This is the second tutorial on how get MiMiCPy running, and deals with setting up the protein+ligand system. Please refer to the previous tutorial on setting up the host before reading this.

## Loading Proteins

Once the host is set-up, information about the system to be simulated needs to be given. Currently, only protein systems with/without ligands are supported. Loading a protein into MiMiCPy is as simple as just feeding it the PDB data. If the PDB data is already available in memory, it can be passed directly to the `mimicpy.Protein` class constructor as a string.

In [1]:
import mimicpy
pdb = ("ATOM      1  N   HIS A   1      49.668  24.248  10.436  1.00 25.00           N\n"
    "ATOM      2  CA  HIS A   1      50.197  25.578  10.784  1.00 16.00           C\n"
    "ATOM      3  C   HIS A   1      49.169  26.701  10.917  1.00 16.00           C\n"
    "ATOM      4  O   HIS A   1      48.241  26.524  11.749  1.00 16.00           O\n"
    "ATOM      5  CB  HIS A   1      51.312  26.048   9.843  1.00 16.00           C\n"
    "ATOM      6  CG  HIS A   1      50.958  26.068   8.340  1.00 16.00           C\n"
    "ATOM      7  ND1 HIS A   1      49.636  26.144   7.860  1.00 16.00           N\n"
    "ATOM      8  CD2 HIS A   1      51.797  26.043   7.286  1.00 16.00           C\n"
    "ATOM      9  CE1 HIS A   1      49.691  26.152   6.454  1.00 17.00           C\n"
    "ATOM     10  NE2 HIS A   1      51.046  26.090   6.098  1.00 17.00           N\n")
prt = mimicpy.Protein(pdb, 'Test_Protein')

Creating protein Test_Protein..
Extracting water..
Extracting amino acid residues..


Here pdb is the PDB string, and the second parameter is the name of the protein used soley for book-keeping purposes. Only amino acid residues and water molecules are extracted. These can be accessed from `prt.pdb` and `prt.water`.

In [2]:
print(prt.pdb)

TITLE     Test_Protein
ATOM      1  N   HIS A   1      49.668  24.248  10.436  1.00 25.00           N
ATOM      2  CA  HIS A   1      50.197  25.578  10.784  1.00 16.00           C
ATOM      3  C   HIS A   1      49.169  26.701  10.917  1.00 16.00           C
ATOM      4  O   HIS A   1      48.241  26.524  11.749  1.00 16.00           O
ATOM      5  CB  HIS A   1      51.312  26.048   9.843  1.00 16.00           C
ATOM      6  CG  HIS A   1      50.958  26.068   8.340  1.00 16.00           C
ATOM      7  ND1 HIS A   1      49.636  26.144   7.860  1.00 16.00           N
ATOM      8  CD2 HIS A   1      51.797  26.043   7.286  1.00 16.00           C
ATOM      9  CE1 HIS A   1      49.691  26.152   6.454  1.00 17.00           C
ATOM     10  NE2 HIS A   1      51.046  26.090   6.098  1.00 17.00           N



In most cases, the PDB will need to be loaded from file. This can be done using the `fromFile()` function.

In [3]:
prt = mimicpy.Protein.fromFile('test.pdb')

test.pdb found!
Creating protein test..
Extracting water..
Extracting amino acid residues..


## Loading Ligands

### Standard Ligands

Loading proteins from PDB files are simple enough, but what if your system consists of ligands as well? Standard ligands, i.e., ligands that contain residues for which the topology information is available in the force field being applied, are simple to add. For example, glycerol is supported by the Amber force fiedl in Gromacs. Oly SDF data from the RCSB PDB database is supported as these provide accurate atom name, bonding, and other information. We can load it from either from an SDF file or SDF data in memory.

In [4]:
glyc = mimicpy.StdLigand.fromFile('glycerol.sdf') # mimicpy.fromBlock(sdf) to load from sdf string in memory

Creating standard ligand from glycerol.sdf..
Cleaning sdf..
Converting to pdb using Openbabel
Assigning correct atom names..
Assigning correct chain IDs..


Since, standard ligands can be handeled by Gromacs safely (protonation, topology generation, etc.) no extra work is required. Openbable is used to convert the SDF into a PDB, and it is cleaned up. Also, the atomic symbols are read for each atom; this will be used in creating the QM input file.

The actual PDB data generated can be viewed from `glyc.pdb`. The ligand cannow be added to our protein.

In [5]:
prt.addLigand(glyc)

Adding standard ligand GOL to Test_Protein..


### Non-Standard Ligands

Non Standard Ligands are small molecules for which the force field information is not available. So, the protonation, topology generation will have to be done by MiMiCPy. This can be done by using the `mimicpy.NonStdLigand` class 

In [6]:
ict = mimicpy.NonStdLigand.fromFile('ICT.sdf', pH=7.5) # fromBlock() for loading SDF from memory

Creating non standard ligand from ICT.sdf..
Cleaning sdf..
Converting to pdb using Openbabel
Adding hydrogens at pH=7.5
Assigning correct atom names..
Assigning correct chain IDs..
Generating topology for the ligand..
ICT.prep found!
Running AmberTools parmchk on ICT.prep..
Output saved to params.frcmod..
Running AmberTools LEaP..
Output saved to ICT.prmtop and ICT.inpcrd..
Converting to Gromacs topology using Acpype..
Output saved to ICT_GMX.gro and ICT_GMX.top..
Cleaning Acpype output..
Renaming atoms to avoid conflict..
Writing position restraint data..
Matching atom order in pdb to itp..
Reading itp atoms..
Renumbering pdb atoms..
Sorting pdb atoms..


As can be seen from the output, there are a lot more steps involved in handling non-standard ligands. Broadly, it is a three step process:

- Protonation: The SDF file is read and protonated at the given pH (default 7) using OpenBabel. The PDB file is cleaned and atomic symbols are read as in the case of standard ligands.
- Topology: The topology (.itp Gromacs file) is generated. Currently, only the Amber force field is supported. The `ChemCompId` tag in the SDF file is taken as the molecule name, and Amber prep and/or frcmod files with this name is searched in the current directory. Using the programs available from AmberTools (parmchk and tleap), the Amber topologies are generated which are then converted to Gromacs topologies using Acpype. The position restratin file for the ligand is also written.
- Final clean up: This includes, among other steps, reordering the atoms in the pdb and itp to match.

If a prep file is not available for a molecules, it can be generated by performing a Gaussian HF/DFT minimization to the molecule and then using AmberTools Antechamber to generate the charges (using RESP scheme) and prep. Also, if the prep file has different atom names to the one in the PDB, a conversion map in the form of a python dictonary can be provided `fromFile(sdf, prep2pdb={'C1':'CA', 'H1':HA'})`. For more details on utility functions like these, please refer to the documentation.

The generated pdb and itp data can be checked for debugging.

In [7]:
print(ict.pdb[:485])

HETATM    1   O1 ICT A   1      56.308 -33.800  17.235  1.00  0.00           O  
HETATM    2   C1 ICT A   1      55.564 -34.058  18.281  1.00  0.00           C  
HETATM    3   O2 ICT A   1      55.359 -35.162  18.693  1.00  0.00           O1-
HETATM    4   C2 ICT A   1      54.974 -32.763  18.936  1.00  0.00           C  
HETATM    5   O3 ICT A   1      53.871 -28.270  19.456  1.00  0.00           O  
HETATM    6   H2 ICT A   1      53.453 -33.722  19.735  1.00  0.00           H  


In [8]:
print(ict.itp[:572])



[ atomtypes ]
;name   bond_type     mass     charge   ptype   sigma         epsilon       Amb
 ICT_o       ICT_o          0.00000  0.00000   A     2.95992e-01   8.78640e-01 ; 1.66  0.2100
 ICT_c       ICT_c          0.00000  0.00000   A     3.39967e-01   3.59824e-01 ; 1.91  0.0860
 ICT_c3       ICT_c3          0.00000  0.00000   A     3.39967e-01   4.57730e-01 ; 1.91  0.1094
 ICT_oh       ICT_oh          0.00000  0.00000   A     3.06647e-01   8.80314e-01 ; 1.72  0.2104
 ICT_ho       ICT_ho          0.00000  0.00000   A     0.00000e+00   0.00000e+00 ; 0.00  0.0000



And finally, the ligand can be added to the protein just like in the dtandard ligand case.

In [9]:
prt.addLigand(ict)

Adding non standard ligand ICT to Test_Protein..


## Preparing the System Topology

Once the protein and ligands are loaded, we can preapre the system for classical MD. At this point, we can choose to start execution on the remote server, which has Gromacs and CPMD installed.

In [10]:
mimicpy.setHost('claix16:tests', 'source gromacs.sh')
mimicpy.setEnv(gmx='gmx_mpi')

Setting remote machine claix16 as host..
Setting current directory to test_..


To start preparation, we have to create the `prepare.MM` handle. Handles are explained more in-depth in the next tutorial, but in brief they control the execution of Gromacs/CPMD/MiMiC jobs. The `prepare.MM` handle can be used to prepare a system for classical MD, and is the first set towards running a MiMiC simulation.

In [11]:
prep = mimicpy.prepare.MM()
prep.getTopology(prt)

Attaching protein 3INM_fixed..
Preparing protein topology..
Writing 3INM_fixed pdb to confin.pdb..
Reading histidine protonation states from list..
Running Gromacs pdb2gmx..
Adding non-standard residues to structure..
Combining ligands topology into single file ligands.itp..
Writing position restraint file for ICT
ligands.itp added to topol.top
Topology prepared..
Saving session to _status.yaml


The `getTopology()` function generates the topology of the total system. It does the following steps:
- Executes gromacs pdb2gmx on the pure protein (+standard residues) to get the pure protein topology
- Adds the non-standard ligand pdbs and topologies to the protein in the right order
- Writes the position restratint file
- Writes a mimicpy topology file (for more details refer Tutorial 4)

The function takes the protein (with all ligands added) as input. Custom parameters for pdb2gmx can be set calling the `topolParams()` function. For more details, refer the documentation.

The next step is to prepare the simulation box. This is done by calling the `prepareBox()` function.

In [12]:
prep.setIons(pname='NA', nname='CL', neutral='')
prep.prepareBox()
mimicpy.closeHost()

Preparing system box..
Solavting protein..
Running Gromacs editconf..
Running Gromacs solvate..
         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.
Adding ions to neutralize charge..
Running Gromacs grompp..
NOTE 1 [file ions.mdp]:
  You are using a plain Coulomb cut-off, which might produce artifacts.
  You might want to consider using PME electrostatics.
Running Gromacs genion..
Simulation box prepared..
Saving session to _status.yaml


This function does the following steps:
- Executes gromacs editconf to resize the box
- Executes gromacs solvate to add waters to the sytem
- Executes genion to add ions to the system

The parameters for each can be set calling `boxParams()`, `solavteParams()`, and `ionParams()`.